# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

### Importação de Bibliotecas e set de chrome ems 2º plano

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.headless = True 
navegador = webdriver.Chrome(options=chrome_options)

navegador.get("https://www.google.com/")

# Passo 1: Pegar a cotação do Dólar
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 

print(cotacao_dolar)

# Passo 2: Pegar a cotação do Euro
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_euro)

# Passo 3: Pegar a cotação do Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")

print(cotacao_ouro)








4.8815
5.170728875
289.47


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados desatualizada

In [2]:
#Importar a lista de produtos
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [3]:
# Agora quero saber qual parte da coluna "Moeda", para assim transformar suas cotações em valores atuais
# Localização -> tabela["Moeda"] == "Dólar"   (Lê-se: Coluna "Moeda" na tabela, em que houver Dólar)

# tabela.loc é uma localização na tabela, de linha e coluna. No caso, a linha será a parte da coluna moeda com dólar.
# A coluna em questão é a "Cotação"
# Por fim, disse que esses valores a partir de agora serão chamados de cotacao_dolar e em float.
# Ou seja, serão os números decimais da cotação atual.

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# Adição de novas colunas à tabela
# Preço de Compra é igual ao Preço Origual multiplicado pela cotação atualizada

tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# Preço de Venda é igual ao Preço de Compra multiplicado pela Margem de lucro

tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,4.881500,4881.451185,1.40,6834.031659
1,Carro Renault,4500.00,Euro,5.170729,23268.279937,2.00,46536.559875
2,Notebook Dell,899.99,Dólar,4.881500,4393.301185,1.70,7468.612015
3,IPhone,799.00,Dólar,4.881500,3900.318500,1.70,6630.541450
4,Carro Fiat,3000.00,Euro,5.170729,15512.186625,1.90,29473.154587
5,Celular Xiaomi,480.48,Dólar,4.881500,2345.463120,2.00,4690.926240
6,Joia 20g,20.00,Ouro,289.470000,5789.400000,1.15,6657.810000


### Exportando a nova base de preços atualizada

In [4]:
# Importação da tabela para o Excel, em arquivo .xlsx, com a retirada do índice  lateral com números. (index=False)

tabela.to_excel("Produtos Atualizado.xlsx", index= False)
